In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta 
import time
import requests
import pickle

In [132]:
start_dt = df['datetime'].iloc[0]

get_weather = []

for hour_idx in range(100000):
    this_dt = start_dt + timedelta(hours = hour_idx)
    
    last_dt, color, time_until_next, time_from_prev = get_flag_color(this_dt)
    #time_until_next < timedelta(days = 3)
    if type(color) == str and time_from_prev < timedelta(days = 2):
        #print(color, time_from_prev, time_until_next)
        get_weather.append([this_dt,color])

IndexError: single positional indexer is out-of-bounds

In [137]:
len(get_weather)

7276

In [3]:
#with open('times_and_colors.pkl', 'wb') as handle:
#    pickle.dump(get_weather, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('times_and_colors.pkl', 'rb') as handle:
    times_and_colors = pickle.load(handle)

In [5]:
counter = 0

for time,color in enumerate(times_and_colors, start=0):
    if color == 'red':
        counter += 1
print(counter)

0


In [7]:
counter = 0

for count, item in enumerate(times_and_colors, start=0):
    time, color = item
    if color == 'yellow':
        #print(count)
        pass

In [57]:
## Dates to weather

In [8]:
idx = 6758
times_and_colors[idx]

[Timestamp('2017-06-12 07:00:00'), 'yellow']

In [18]:
this_line = times_and_colors[idx]
weather_dict,weather,unix_time = get_weather_from_dt(this_line)

In [19]:
weather_dict

{'timestamp': Timestamp('2017-06-12 07:00:00'),
 'color': 'yellow',
 'summary': 'Partly Cloudy',
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 56.43,
 'windSpeed': 9.21,
 'visibility': 6.216}

In [163]:
## Get all weather

In [20]:
for count, item in enumerate(times_and_colors, start=0):
    if count > 5:
        break
    print(count,item)
    weather_dict,weather,unix_time = get_weather_from_dt(item)

0 [Timestamp('2009-11-15 09:00:00'), 'yellow']
1 [Timestamp('2009-11-15 10:00:00'), 'yellow']
2 [Timestamp('2009-11-15 11:00:00'), 'green']
3 [Timestamp('2009-11-15 12:00:00'), 'green']
4 [Timestamp('2009-11-15 13:00:00'), 'green']
5 [Timestamp('2009-11-15 14:00:00'), 'green']


In [235]:
df_tweets_0_1000 = pd.DataFrame()

for count, item in enumerate(times_and_colors, start=0):
    if count > 1000:
        break
    
    if count % 100 == 0:
        print(count)
    
    try:
        weather_dict,weather,unix_time = get_weather_from_dt(item)
        df_tweets_0_1000 = df_tweets_0_1000.append(weather_dict,ignore_index=True)
    except:
        print(f'did pass on {count}')

df_tweets_0_1000.to_csv('df_tweets_0_1000.csv',index=False)

0
did pass on 91
100
200
did pass on 209
did pass on 210
300
400
did pass on 437
500
did pass on 582
600
did pass on 655
700
800
900
1000


In [38]:
df_tweets_6000_7000 = pd.DataFrame()

for count, item in enumerate(times_and_colors, start=0):
    if count > 6000:
        if count > 7000:
            break

        if count % 100 == 0:
            print(count)

        try:
            weather_dict,weather,unix_time = get_weather_from_dt(item)
            df_tweets_6000_7000 = df_tweets_6000_7000.append(weather_dict,ignore_index=True)
        except:
            print(f'did pass on {count}')

df_tweets_6000_7000.to_csv('csvs/df_tweets_6000_7000.csv',index=False)

6100
6200
6300
6400
6500
6600
6700
6800
6900
7000


In [12]:
## Functions

with open('keys/my_dark_sky_key.pkl', 'rb') as handle:
    dark_sky_key = pickle.load(handle)
    
lat_num,lng_num = 52.213268, 0.123658
lat,lng = str(lat_num),str(lng_num)
    
def get_weather_from_dt(line):
    this_timestamp, color = line
    
    # make url
    unix_time = str(int(time.mktime(this_timestamp.timetuple())))
    url = f'https://api.darksky.net/forecast/{dark_sky_key}/{lat},{lng},{unix_time}'
    
    # get weather from timestamp
    r = requests.get(url)
    weather = r.json()
    
    # parse response
    currently = weather['currently']

    weather_dict = dict(
                        timestamp = this_timestamp,
                        color = color,
                        summary = currently['summary'],
                        precipIntensity = currently['precipIntensity'],
                        precipProbability = currently['precipProbability'],
                        temperature = currently['temperature'],
                        windSpeed = currently['windSpeed'],
                        visibility = currently['visibility'])
    
    return weather_dict,weather,unix_time

In [46]:
def get_flag_color(dt):
    idx = 0
    this_dt = df['datetime'].iloc[idx]
    while dt > this_dt:
        idx += 1
        this_dt = df['datetime'].iloc[idx]
    last_dt = df['datetime'].iloc[idx - 1]
    color = df['color'].iloc[idx - 1]
    
    next_time = df['datetime'].iloc[idx]
    
    time_from_prev = dt - last_dt
    time_until_next = next_time - dt
    
    return last_dt, color, time_until_next, time_from_prev

In [2]:
#pd.read_csv('df.csv')